In [1]:
VERSION = 'test'

%load_ext tensorboard

from model.model import *
from process_func import *
import tensorflow as tf
DATA_DIR = f'datasets/'
map = {'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Major': 'Other', 
       'Col': 'Other', 'Dr' : 'Other', 'Rev' : 'Other', 'Capt': 'Other', 
       'Jonkheer': 'Royal', 'Sir': 'Royal', 'Lady': 'Royal', 
       'Don': 'Royal', 'Countess': 'Royal', 'Dona': 'Royal'}
titles = ['Miss', 'Mr', 'Mrs', 'Royal', 'Other', 'Master']

# Run for validation

In [2]:
data = Titanic(DATA_DIR, show_head = False)
data.Preprocess(map, titles)
dataset = data.GetXandY()
# Run for validation

Initializing...
Data Loaded.
Data Preprocessing...
Done Preprocessing.
Returned Data Dictionary


In [3]:
model = BinaryClassification(dataset['x_train'].shape)
CP_dir = SetCheckpoint(VERSION)
TB_dir = SetLog(VERSION)
# Run for validation

Model: "TitanicBC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 15)]              0         
                                                                 
 Dense0 (Dense)              (None, 13)                208       
                                                                 
 dropout (Dropout)           (None, 13)                0         
                                                                 
 Dense1 (Dense)              (None, 8)                 112       
                                                                 
 Output (Dense)              (None, 1)                 9         
                                                                 
Total params: 329
Trainable params: 329
Non-trainable params: 0
_________________________________________________________________


In [5]:
BATCH_SIZE = 64
EPOCHS = 100
callbacks = DefCallbacks(VERSION, CP = CP_dir, TB = TB_dir)
model.fit(dataset['x_train'], dataset['y_train'], callbacks = callbacks, 
          validation_split = 0.2, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 2)

Epoch 1/100
12/12 - 1s - loss: 0.6805 - binary_accuracy: 0.5351 - val_loss: 0.6316 - val_binary_accuracy: 0.6872 - 1s/epoch - 95ms/step
Epoch 2/100
12/12 - 0s - loss: 0.6405 - binary_accuracy: 0.6559 - val_loss: 0.5998 - val_binary_accuracy: 0.7374 - 74ms/epoch - 6ms/step
Epoch 3/100
12/12 - 0s - loss: 0.6126 - binary_accuracy: 0.7107 - val_loss: 0.5706 - val_binary_accuracy: 0.7877 - 72ms/epoch - 6ms/step
Epoch 4/100
12/12 - 0s - loss: 0.5924 - binary_accuracy: 0.7247 - val_loss: 0.5451 - val_binary_accuracy: 0.7933 - 65ms/epoch - 5ms/step
Epoch 5/100
12/12 - 0s - loss: 0.5737 - binary_accuracy: 0.7753 - val_loss: 0.5213 - val_binary_accuracy: 0.7989 - 68ms/epoch - 6ms/step
Epoch 6/100
12/12 - 0s - loss: 0.5616 - binary_accuracy: 0.7767 - val_loss: 0.5012 - val_binary_accuracy: 0.7933 - 71ms/epoch - 6ms/step
Epoch 7/100
12/12 - 0s - loss: 0.5431 - binary_accuracy: 0.7823 - val_loss: 0.4831 - val_binary_accuracy: 0.7933 - 72ms/epoch - 6ms/step
Epoch 8/100
12/12 - 0s - loss: 0.5302 - bi

In [4]:
model.load_weights(CP_dir)
pred = model.predict(dataset['x_test'], batch_size=200)
pred = tf.reshape(pred, (-1))
data_check =  pd.read_csv(DATA_DIR + "gender_submission.csv")
pred_series = pd.Series(pred)
print(pred_series[:10])
# Creating new column of predictions in data_check dataframe
data_check['check'] = pred_series
series = []
for val in data_check.check:
    if val >= 0.5:
        series.append(1)
    else:
        series.append(0)
data_check['final'] = series
match = 0
nomatch = 0
for val in data_check.values:
    if val[1] == val[3]:
        match += 1
    else:
        nomatch += 1

print(f'Accuracy : {match/data_check.shape[-2] * 100 : .2f} %')
# Run for validation

NameError: name 'CP_dir' is not defined

In [7]:
temp = pd.DataFrame(pd.read_csv(DATA_DIR + "test.csv")['PassengerId'])
temp['Survived'] = data_check['final']
temp.to_csv(f"submission/submission_{VERSION}.csv", index = False)
# Run for validation

In [8]:
# Family size from Family Name
# apply scaler on Age, Fare?
# Title (Mr. Miss. ) - Married?
# Social class from Ticket class and Fare